# Matrix Factorization-based Method

먼저 필요한 라이브러리를 설치해야 한다.<br><br>
사전 설치가 필요한 Library 리스트<br>
- pip install pandas: 데이터프레임 처리 라이브러리<br>
- pip install numpy: array 처리 라이브러리<br>
- pip install suprise: 추천시스템 모델링 라이브러리<br>
- pip install scipy: 계산 라이브러리

분석에 필요한 라이브러리를 불러온다.

In [1]:
import pandas as pd
import numpy as np
import surprise
import scipy

<br>

# 1. SVD를 활용한 도서 추천

## 데이터 처리

이 실습에서는 SVD를 활용해 도서를 추천한다.<br><br>
Book crossing 데이터를 기반으로 도서 추천을 진행한다.<br>
추천에 필요한 평점, 도서 정보를 불러와 데이터프레임을 구성하며, 각 데이터는 아래의 정보를 포함하고 있다.<br>
- BX-Users.csv: UserID, Location, Age
- BX-Book-Ratings.csv: UserID, BookID, Rating

In [2]:
ratings_df = pd.read_csv('./data/BX-CSV-Dump/BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding='latin-1')
ratings_df.columns = ['UserID', 'BookID', 'Rating']

books_df = pd.read_csv('./data/BX-CSV-Dump/BX-Books.csv', sep=';', error_bad_lines=False, encoding='latin-1')

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:30

평점 데이터프레임은 다음과 같다.<br>
예를 들어 276725번 user는 034545104X번 도서에 평점 0점을 주었다.

In [3]:
ratings_df.head()

,UserID,BookID,Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


도서 데이터프레임은 다음과 같다.<br>
예를 들어 0195153448 도서의 제목은 Classical Mythology, 작가는 Mark P. O. Morford이며, 아래와 같이 추가적인 정보들이 담겨있다.

In [4]:
books_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


도서 데이터프레임에서 필요한 'BookID', 'BookTitle', 'BookAuthor' 정보만 뽑아서 최종적인 데이터프레임을 구축한다.

In [5]:
books_df = books_df.iloc[:,:3]
books_df.columns = ['BookID', 'BookTitle', 'BookAuthor']
books_df.head()

,BookID,BookTitle,BookAuthor
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


추천 시스템 라이브러리 surprise를 사용하기 위해 구축한 ratings_df 데이터프레임을 해당 라이브러리에서 요구하는 input 형태에 맞게 변형한다.<br>
surprise 라이브러리는 dataset을 인풋으로 받으며, pandas 데이터프레임으로부터 dataset을 로딩하기 위해 Reader object와 load_from_df() method를 사용한다.

In [6]:
from surprise import Reader
from surprise import Dataset

In [7]:
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(ratings_df, reader)

<br>

## SVD 모델링

SVD 모델링에 필요한 라이브러리를 불러온다.

In [8]:
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise import accuracy

<br>

surprise.SVD의 주요 옵션은 아래와 같다.<br>
- n_factors: factor의 개수. 디폴트는 100.
- n_epochs: SGD 학습 반복 횟수. 디폴트는 20.
- lr_all: SGD 학습 중 모든 파라메터에 대한 learning rate. 디폴트는 0.005.

SVD의 factor의 개수와 learning rate 옵션의 조합을 이용해 아래와 같이 4가지 옵션 리스트를 생성한다.

In [9]:
options = [{'n_factors': 100, 'lr_all': 0.005},
           {'n_factors': 100, 'lr_all': 0.001},
           {'n_factors': 200, 'lr_all': 0.005},
           {'n_factors': 200, 'lr_all': 0.001}]

위에서 생성한 4가지 옵션을 기반으로 4가지 모델을 만들고 cross-validation 결과를 통해 가장 성능이 좋은 모델을 선택한다.

In [10]:
def model_selection(options):
    perf = []
    # iterate over all algorithms
    for i, option in enumerate(options):
        print('==> ', i + 1, '번째 모델 학습을 시작합니다.')
        
        # make SVD model for each option
        algo = SVD(n_factors=option['n_factors'], lr_all=option['lr_all'])

        # perform cross validation
        results = cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

        # get results
        tmp = pd.DataFrame.from_dict(results).mean(axis=0)
        tmp = tmp.append(pd.Series([option['n_factors'], option['lr_all']], index=['n_factors', 'lr']))
        perf.append(tmp)

    model_selection_results = pd.DataFrame(perf).set_index(['n_factors', 'lr']).sort_values('test_rmse')
    return model_selection_results

In [11]:
model_selection_results = model_selection(options)

==>  1 번째 모델 학습을 시작합니다.
==>  2 번째 모델 학습을 시작합니다.
==>  3 번째 모델 학습을 시작합니다.
==>  4 번째 모델 학습을 시작합니다.


In [12]:
model_selection_results

test_rmse   fit_time  test_time
n_factors lr                                    
100.0     0.001   3.454918  46.999472   4.401639
200.0     0.001   3.459197  77.047175   4.183278
          0.005   3.470511  76.553240   3.986515
100.0     0.005   3.499080  47.444060   4.063514

<br>

## 최적 SVD 모델 분석

위에서 실험한 4가지 모델 중 test 데이터에 대한 rmse 값을 기준으로 n_factors = 100, lr_all = 0.001인 SVD 알고리즘이 가장 좋은 결과를 도출했다.<br>
따라서 해당 알고리즘을 이용해 보다 자세한 결과를 확인하기 위해 데이터를 7:3의 비율로 train과 test 데이터로 분할한다.

In [13]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.3, random_state=42)

앞서 model selection에서 선택된 n_factors = 100, lr_all = 0.001인 SVD 알고리즘을 구축하고 해당 모델을 train 데이터로 학습한다.

In [14]:
model_svd = SVD(n_factors=100, lr_all=0.001)
model_svd.fit(trainset)

train 데이터로 학습한 SVD 알고리즘을 기반으로 test 데이터의 평점을 예측한다.

In [15]:
preds = model_svd.test(testset)
preds

[Prediction(uid=273718, iid='0711705577', r_ui=5.0, est=4.311029904099282, details={'was_impossible': False}),
 Prediction(uid=35859, iid='0451452143', r_ui=0.0, est=0.993215945677755, details={'was_impossible': False}),
 Prediction(uid=127200, iid='0553095161', r_ui=0.0, est=1.141603445400861, details={'was_impossible': False}),
 Prediction(uid=2024, iid='0802130259', r_ui=5.0, est=4.449896656659437, details={'was_impossible': False}),
 Prediction(uid=275970, iid='0312275366', r_ui=0.0, est=0.5642438906739597, details={'was_impossible': False}),
 Prediction(uid=125039, iid='0670555061', r_ui=0.0, est=1.1381957466166348, details={'was_impossible': False}),
 Prediction(uid=239584, iid='0066214440', r_ui=0.0, est=4.328452849898262, details={'was_impossible': False}),
 Prediction(uid=70052, iid='0743406184', r_ui=0.0, est=0.873752365822138, details={'was_impossible': False}),
 Prediction(uid=131402, iid='0451203070', r_ui=0.0, est=1.1842671539623804, details={'was_impossible': False}),
 P

In [16]:
accuracy.rmse(preds)

RMSE: 3.4555


3.4554724800393886

test 데이터 중 예측값과 실제값의 차이가 작은 상위 10개의 best prediction을 보다 상세하게 살펴보면 다음과 같다.

In [17]:
preds_df = pd.DataFrame(preds)
preds_df.head()

,uid,iid,r_ui,est,details
0,273718,0711705577,5.0,4.311030,{'was_impossible': False}
1,35859,0451452143,0.0,0.993216,{'was_impossible': False}
2,127200,0553095161,0.0,1.141603,{'was_impossible': False}
3,2024,0802130259,5.0,4.449897,{'was_impossible': False}
4,275970,0312275366,0.0,0.564244,{'was_impossible': False}


preds_df 데이터프레임에 예측값과 실제값의 차이를 나타내는 err 열을 추가한다.

In [18]:
preds_df['err'] = abs(preds_df.est - preds_df.r_ui)
preds_df.head()

,uid,iid,r_ui,est,details,err
0,273718,0711705577,5.0,4.311030,{'was_impossible': False},0.688970
1,35859,0451452143,0.0,0.993216,{'was_impossible': False},0.993216
2,127200,0553095161,0.0,1.141603,{'was_impossible': False},1.141603
3,2024,0802130259,5.0,4.449897,{'was_impossible': False},0.550103
4,275970,0312275366,0.0,0.564244,{'was_impossible': False},0.564244


err를 기준으로 실제값의 차이가 작은 상위 10개의 best prediction을 살펴본다.

In [19]:
best_preds = preds_df.sort_values(by='err')[:10]
best_preds

,uid,iid,r_ui,est,details,err
321427,232131,0060924322,0.0,0.0,{'was_impossible': False},0.0
157476,73394,1551665891,0.0,0.0,{'was_impossible': False},0.0
117953,242824,0345334531,0.0,0.0,{'was_impossible': False},0.0
241095,198711,0671021060,0.0,0.0,{'was_impossible': False},0.0
98400,110973,0425142485,0.0,0.0,{'was_impossible': False},0.0
72944,98741,0525937579,0.0,0.0,{'was_impossible': False},0.0
72922,73394,0439443857,0.0,0.0,{'was_impossible': False},0.0
83592,198711,0671882902,0.0,0.0,{'was_impossible': False},0.0
341876,198711,0380763389,0.0,0.0,{'was_impossible': False},0.0
52728,114414,0671749412,0.0,0.0,{'was_impossible': False},0.0


best prediction 중 1번째 user가 실제로 봤던 도서와 추천받은 도서를 비교하면 아래와 같다.<br>
먼저 1번째 user가 실제로 봤던 도서의 제목과 저자를 살펴본다.

In [20]:
user_BookID = ratings_df[ratings_df['UserID'] == list(best_preds.uid)[0]]
user_BookID = user_BookID['BookID']
user_BookID = list(user_BookID)

In [21]:
user_Book = books_df[books_df['BookID'].isin(user_BookID)]
user_Book

,BookID,BookTitle,BookAuthor
18,0440234743,The Testament,John Grisham
26,0971880107,Wild Animus,Rich Shapero
28,0345417623,Timeline,MICHAEL CRICHTON
31,0425163091,Chocolate Jesus,Stephan Jaramillo
38,0449005615,Seabiscuit: An American Legend,LAURA HILLENBRAND
47,0425182908,Isle of Dogs,Patricia Cornwell
51,0842342702,Left Behind: A Novel of the Earth's Last Days ...,Tim Lahaye
52,0440225701,The Street Lawyer,JOHN GRISHAM
56,0380715899,A Soldier of the Great War,Mark Helprin
66,042511774X,Breathing Lessons,Anne Tyler


1번째 user가 추천받은 영화의 제목과 장르를 확인하고 실제 데이터와 비교해본다.<br>

In [22]:
recommended_Book = books_df[books_df['BookID'] == list(best_preds.iid)[0]]
recommended_Book

,BookID,BookTitle,BookAuthor
110526,0060924322,When Did Wild Poodles Roam the Earth? An Impon...,David Feldman


<br>

# 2. SVD 추천 알고리즘 구현

## 데이터 처리

surprise.SVD 라이브러리에서 제공하는 SVD 기반의 추천 알고리즘이 아닌 Scipy 라이브러리에서 제공하는 SVD 함수를 이용해 해당 알고리즘을 구현한다.<br>

SVD 함수는 array를 input으로 받기 때문에 원본 데이터를 이에 맞게 변형한다.<br><br>
먼저 원본 데이터 중 일부 (20,000개)에 대해 다음과 같이 x축이 BookID, y축이 UserID인 평점 행렬(rate matrix) 피봇테이블(pivot table)을 만든다.<br>
평점 행렬의 행은 특정 user의 평점이고 평점 행렬의 열은 특정 book의 평점이다.

In [23]:
ratings_df_sample = ratings_df.iloc[:20000, :]
ratings_df_sample.head()

,UserID,BookID,Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [24]:
R_df = ratings_df_sample.pivot(index = 'UserID', columns ='BookID', values = 'Rating').fillna(0)
R_df.head()

BookID,0002005018,0002231115,0002232766,0002240114,000225669X,000254794,0002558122,0002740230,0006128831,0006144500,...,B0000DAPP1,B158991965,B460712002,BCID694577184,DITISEENSOORT,N3453124715,NONFICTION,O6712345670,O76790592X,O809463121
UserID,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


샘플링된 데이터에 속하는 전체 user id를 저장한다.

In [25]:
all_UserID = list(R_df.index)
len(all_UserID)

2180

각 사용자 마다 평균을 구해 데이터를 정규화한다. (de-mean)<br>
정규화를 적용한 결과는 데이터프레임에서 numpy 배열로 변환한다.

In [26]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


<br>

## SVD 모델링

Scipy 라이브러리에서 제공하는 SVD 함수를 이용해 SVD 기반의 추천 알고리즘을 구현한다.<br>
n_factors를 나타내는 옵션 k는 위에서 찾은 best prediction model과 동일하게 100으로 설정한다.

In [27]:
from scipy.sparse.linalg import svds

U, sigma, Vt = svds(R_demeaned, k = 100)

여기서 sigma는 대각 행렬이 아니라 그냥 값만 갖고 있기 때문에, 계산 편의를 위해 대각 행렬로 변환한다.

In [28]:
sigma = np.diag(sigma)

U, Σ, V_transpose의 행렬 곱과 계산 과정을 통해 k = 100을 갖는 R의 근사 행렬을 얻을 수 있다.<br> 
10점 척도의 평점 예측을 위해 각 사용자의 평균을 다시 더해준다.

In [29]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [30]:
preds_df.head()

BookID,0002005018,0002231115,0002232766,0002240114,000225669X,000254794,0002558122,0002740230,0006128831,0006144500,...,B0000DAPP1,B158991965,B460712002,BCID694577184,DITISEENSOORT,N3453124715,NONFICTION,O6712345670,O76790592X,O809463121
0,7.926679e-18,7.222441e-18,7.222441e-18,7.607859e-18,7.458686e-18,7.489653e-18,7.222441e-18,7.381050e-18,7.222441e-18,7.338094e-18,...,1.631105e-17,7.480378e-18,7.222441e-18,7.222441e-18,7.222441e-18,7.491732e-18,7.222441e-18,7.381050e-18,7.222441e-18,7.476642e-18
1,1.802895e-20,1.646133e-20,1.646133e-20,1.731906e-20,1.698704e-20,1.705596e-20,1.646133e-20,1.681427e-20,1.646133e-20,1.671868e-20,...,3.258458e-20,1.703532e-20,1.646133e-20,1.646133e-20,1.646133e-20,1.706062e-20,1.646133e-20,1.681427e-20,1.646133e-20,1.702705e-20
2,2.936088e-03,2.875593e-03,2.875593e-03,2.908177e-03,2.895432e-03,2.898059e-03,2.875593e-03,2.888874e-03,2.875593e-03,2.885261e-03,...,1.339161e-03,2.897291e-03,2.875593e-03,2.875593e-03,2.875593e-03,2.898337e-03,2.875593e-03,2.888874e-03,2.875593e-03,2.897096e-03
3,-1.831429e-04,-1.069891e-04,-1.069891e-04,-1.486465e-04,-1.324675e-04,-1.358209e-04,-1.069891e-04,-1.240719e-04,-1.069891e-04,-1.194351e-04,...,-5.280967e-05,-1.348257e-04,-1.069891e-04,-1.069891e-04,-1.069891e-04,-1.360878e-04,-1.069891e-04,-1.240719e-04,-1.069891e-04,-1.344677e-04
4,4.428121e-04,4.347553e-04,4.347553e-04,4.391092e-04,4.374100e-04,4.377607e-04,4.347553e-04,4.365335e-04,4.347553e-04,4.360503e-04,...,2.120510e-04,4.376576e-04,4.347553e-04,4.347553e-04,4.347553e-04,4.377950e-04,4.347553e-04,4.365335e-04,4.347553e-04,4.376282e-04


각 사용자에 대한 예측 행렬을 이용하여 각 사용자에게 도서를 추천해 주는 함수를 만들 수 있다.<br> 
이 함수는 특정 user가 이전에 평점을 주지 않은 도서의 평점을 예측하고, 예측한 평점 순으로 도서를 추천하는 역할을 한다.<br>
추가적으로 각 도서의 특징 정보 (제목, 저자)가 없기 때문에 예측 행렬에 해당 정보들을 병합해준다.

In [31]:
def recommend_books(preds_df, UserID, books_df, ratings_df, all_UserID, num_recommendations=5):
    # Get and sort the user's predictions
    user_row_number = all_UserID.index(UserID)
    sorted_user_preds = preds_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the book information.
    user_data = ratings_df[ratings_df.UserID == UserID]
    user_full = user_data.merge(books_df, how = 'left', left_on = 'BookID', right_on = 'BookID').sort_values(['Rating'], ascending=False)

    print('User {0} has already rated {1} books.'.format(UserID, user_full.shape[0]))
    print('Recommending the highest {0} predicted ratings books not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating books that the user hasn't seen yet.
    recommendations = (books_df[~books_df['BookID'].isin(user_full['BookID'])].
                       merge(pd.DataFrame(sorted_user_preds).reset_index(), 
                             how = 'left', left_on = 'BookID', right_on = 'BookID').
                       rename(columns = {user_row_number: 'Preds'}).
                       sort_values('Preds', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

sample 데이터 중 한 user (UserID = 2,977)에 대해 10개의 도서를 추천한 결과와 실제 user가 읽은 도서를 비교하면 아래와 같다.

In [32]:
already_rated, preds = recommend_books(preds_df, 2977, books_df, ratings_df, all_UserID, 10)

User 2977 has already rated 232 books.
Recommending the highest 10 predicted ratings books not already rated.


In [33]:
already_rated.head(10)

,UserID,BookID,Rating,BookTitle,BookAuthor
143,2977,0671496107,10,Immortal Poems of the English Language,Oscar Williams
189,2977,0867163968,10,To Live As Francis Lived: A Guide for Secular ...,Leonard Foley
71,2977,0393962873,10,The Norton Anthology of English Literature (No...,M. H. Abrams
117,2977,0520011309,10,Henry VIII,J. J. Scarisbrick
124,2977,055320338X,10,Against Our Will,Susan Brownmiller
171,2977,0781805899,10,Beginners Welsh (Beginner's (Foreign Language)),Heini Gruffudd
184,2977,0819815195,10,NaN,NaN
185,2977,0821221817,10,The National Parks : A Postcard Folio Book,Ansel Adams
187,2977,0843711299,10,The Times atlas of world history,Barracl
54,2977,0361074662,10,Tales from Bohemia,Karel JaromÃ­r Erben


In [34]:
preds

,BookID,BookTitle,BookAuthor
905,0345391802,The Hitchhiker's Guide to the Galaxy,Douglas Adams
1041,0671510053,SHIPPING NEWS,Annie Proulx
1898,0684829746,CRUDDY : An Illustrated Novel,Lynda Barry
10109,0553573616,My Point...And I Do Have One,ELLEN DEGENERES
15642,0553272586,Farewell to Manzanar: A True Story of Japanese...,Jeanne W. Houston
1061,0743237188,Fall On Your Knees (Oprah #45),Ann-Marie MacDonald
480,0805063897,Nickel and Dimed: On (Not) Getting By in America,Barbara Ehrenreich
1983,067169071X,EVERYTHING SHE EVER WANTED,Ann Rule
3600,0525946241,Picture Maker,Penina Keen Spinka
162686,0767904311,Kindred Spirits : How the Remarkable Bond Betw...,ALLEN M. DVM MS SCHOEN
